In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from collections import OrderedDict
import torch
torch.manual_seed(0)
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
from abcd.local.paths import output_path
from abcd.data.read_data import get_subjects_events_sf, add_event_vars, add_subject_vars, filter_events, filter_subjects
import abcd.data.VARS as VARS
from abcd.data.define_splits import SITES, save_restore_sex_fmri_splits
from abcd.data.divide_with_splits import divide_events_by_splits
from abcd.data.var_tailoring.normalization import normalize_var
from abcd.data.pytorch.get_dataset import PandasDataset
from abcd.models.classification.FullyConnected import FullyConnected3
from abcd.training.ClassifierTrainer import ClassifierTrainer
from abcd.local.paths import core_path, output_path
from abcd.data.VARS import STRUCT_FILES, STRUCT_FEATURES

In [3]:
# Fetch subjects and events
subjects_df, events_df = get_subjects_events_sf()
# Change to 0 (male) and 1 (female)
events_df.loc[events_df["kbi_sex_assigned_at_birth"] == 1.0, "kbi_sex_assigned_at_birth"] = 0
events_df.loc[events_df["kbi_sex_assigned_at_birth"] == 2.0, "kbi_sex_assigned_at_birth"] = 1
# Leave only the baseline visits
events_df = events_df.loc[(events_df['eventname'] == 'baseline_year_1_arm_1')]
print("Leaving baseline visits, we have {} events".format(len(events_df)))

Leaving baseline visits, we have 9086 events


In [4]:
# Define target and features
target_col = 'kbi_sex_assigned_at_birth'
labels = ["Male", "Female"]
feature_cols = list(VARS.NAMED_CONNECTIONS.keys()) + list(VARS.STRUCT_FEATURES.keys())

# Print distribution of baseline class
for val in set(events_df['kbi_sex_assigned_at_birth']):
    print('{} visits with {} target'.format(len(events_df.loc[events_df["kbi_sex_assigned_at_birth"] == val]), labels[int(val)]))

4704 visits with Male target
4382 visits with Female target


In [5]:
# Normalize features
for var_id in feature_cols:
    events_df = normalize_var(events_df, var_id, var_id)

In [6]:
# Divide events into training, validation and testing
splits = save_restore_sex_fmri_splits(k=5)
ood_site_id = SITES[0]
events_train, events_id_test, events_ood_test = divide_events_by_splits(events_df, splits, ood_site_id)
print("Nr. events train: {}, val: {}, test: {}".format(len(events_train), len(events_id_test), len(events_ood_test)))

Nr. events train: 7064, val: 1738, test: 284


In [7]:
# Define PyTorch datasets and dataloaders
datasets = OrderedDict([('train', PandasDataset(events_train, feature_cols, target_col)),
            ('val', PandasDataset(events_id_test, feature_cols, target_col)),
            ('test', PandasDataset(events_ood_test, feature_cols, target_col))])

In [8]:
# Create dataloaders
batch_size = 64
dataloaders = OrderedDict([(dataset_name, DataLoader(dataset, batch_size=batch_size, shuffle=True))
    for dataset_name, dataset in datasets.items()])

for X, y in dataloaders['train']:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X: torch.Size([64, 177])
Shape of y: torch.Size([64]) torch.int64


In [9]:
# Determine device for training
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device


In [10]:
# Define model
models_path = os.path.join(output_path, 'ABCD_sex_prediction_sf', 'models')
model = FullyConnected3(save_path=models_path, labels=labels, input_size=len(feature_cols))
model = model.to(device)
print(model)

FullyConnected3(
  (softmax): Softmax(dim=1)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_layers): Sequential(
    (0): Linear(in_features=177, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [11]:
# Define optimizer and trainer
learning_rate = 1e-3
loss_f = nn.CrossEntropyLoss()
trainer_path = os.path.join(output_path, 'ABCD_sex_prediction_sf', 'results')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
trainer = ClassifierTrainer(trainer_path, device, optimizer, loss_f, labels=labels)

In [12]:
# Train model
trainer.train(model, dataloaders['train'], dataloaders, 
              nr_epochs=150, starting_from_epoch=0,
              print_loss_every=10, eval_every=10, export_every=50, verbose=True)

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 0
train CrossEntropyLoss: 0.693 B-Acc.: 0.518 F1: 0.467
val CrossEntropyLoss: 0.693 B-Acc.: 0.524 F1: 0.470
test CrossEntropyLoss: 0.693 B-Acc.: 0.519 F1: 0.457
Saved PyTorch model state FullyConnected3_epoch0.pth in C:\Users\camgonza\Box\Camila Gonzalez's Files\DATA\ABCD\output\ABCD_sex_prediction_sf\models
Saved trainer state ClassifierTrainer_optimizer_epoch0.pth in C:\Users\camgonza\Box\Camila Gonzalez's Files\DATA\ABCD\output\ABCD_sex_prediction_sf\results\states
Progress stored in C:\Users\camgonza\Box\Camila Gonzalez's Files\DATA\ABCD\output\ABCD_sex_prediction_sf\results


  1%|          | 1/150 [00:03<09:20,  3.76s/it]

Starting epoch 1, loss 0.6646638566309268


  7%|▋         | 10/150 [00:20<05:00,  2.15s/it]

...

Epoch 140
train CrossEntropyLoss: 0.487 B-Acc.: 0.761 F1: 0.756
val CrossEntropyLoss: 0.517 B-Acc.: 0.739 F1: 0.734
test CrossEntropyLoss: 0.528 B-Acc.: 0.727 F1: 0.721


 94%|█████████▍| 141/150 [04:50<00:18,  2.05s/it]

Starting epoch 141, loss 0.47953805246868647


100%|██████████| 150/150 [05:07<00:00,  2.05s/it]



Finished training
Epoch 150
train CrossEntropyLoss: 0.467 B-Acc.: 0.777 F1: 0.776
val CrossEntropyLoss: 0.512 B-Acc.: 0.751 F1: 0.750
test CrossEntropyLoss: 0.521 B-Acc.: 0.730 F1: 0.728
Saved PyTorch model state FullyConnected3_epoch150.pth in C:\Users\camgonza\Box\Camila Gonzalez's Files\DATA\ABCD\output\ABCD_sex_prediction_sf\models
Saved trainer state ClassifierTrainer_optimizer_epoch150.pth in C:\Users\camgonza\Box\Camila Gonzalez's Files\DATA\ABCD\output\ABCD_sex_prediction_sf\results\states
Progress stored in C:\Users\camgonza\Box\Camila Gonzalez's Files\DATA\ABCD\output\ABCD_sex_prediction_sf\results
